# PHASE 2: Exloratory Data Analysis

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# =============================================================================
# 1. DATA INGESTION & TARGET CLASS DISTRIBUTION
# =============================================================================
df = pd.read_excel('cleaned_financial_loan_data.xlsx')

print("--- TARGET CLASS DISTRIBUTION ---")
class_counts = df['loan_status'].value_counts()
class_pct = (class_counts / len(df)) * 100
print(pd.DataFrame({'Count': class_counts, 'Percentage (%)': class_pct}))

# =============================================================================
# 2. FINANCIAL EXPOSURE & BUSINESS IMPACT ANALYSIS
# =============================================================================
# Defining the binary target: 1 for 'Charged Off' (Default), 0 for 'Fully Paid'
df['defaulted'] = (df['loan_status'] == 'Charged Off').astype(int)

# Key Portfolio Metrics
default_rate = df['defaulted'].mean()
avg_loan_size = df['loan_amount'].mean()
portfolio_aum = len(df) * avg_loan_size  # Assets Under Management
lgd_rate = 0.60  # Loss Given Default (Assumption: 60% loss after recovery)

# Estimated Expected Loss (EL) Calculation
estimated_annual_loss = portfolio_aum * default_rate * lgd_rate

print(f"\n💰 PORTFOLIO RISK SUMMARY:")
print(f"   Avg Ticket Size:      ${avg_loan_size:,.2f}")
print(f"   Portfolio Default:    {default_rate*100:.2f}%")
print(f"   Total Exposure (AUM): ${portfolio_aum:,.2f}")
print(f"   Estimated Annual Loss: ${estimated_annual_loss:,.2f}")

# =============================================================================
# 3. UNIVARIATE ANALYSIS: NUMERICAL RISK DRIVERS
# =============================================================================
def analyze_numeric(df, col, target):
    """
    Conducts a detailed statistical audit of continuous variables,
    evaluating distribution symmetry and discrimination power between classes.
    """
    print(f"\n--- NUMERICAL AUDIT: {col.upper()} ---")
    
    # Central Tendency & Dispersion
    print("DESCRIPTIVE STATISTICS:")
    print(df[col].describe().to_frame().T)

    # Shape of Distribution
    print(f"\nDISTRIBUTION SHAPE:")
    print(f"   Skewness: {df[col].skew():.2f}")
    print(f"   Kurtosis: {df[col].kurtosis():.2f}")

    # Discrimination Power (Solvent vs. Defaulted)
    print(f"\nGROUP DISCRIMINATION (by {target}):")
    by_target = df.groupby(target)[col].agg(['mean', 'median', 'std', 'min', 'max'])
    print(by_target)

    # Calculating Variance between Groups
    means = df.groupby(target)[col].mean()
    percent_diff = ((means[1] - means[0]) / means[0]) * 100
    direction = 'Higher' if percent_diff > 0 else 'Lower'
    print(f"\nRESULT: Defaulters exhibit a {abs(percent_diff):.2f}% {direction} {col} than safe borrowers.")

    # MULTI-MODAL VISUALIZATION
    # Visualizing density, outliers, and class-wise distribution spread
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Distribution Overlay
    sns.histplot(df[col], kde=True, ax=axes[0], color='steelblue')
    axes[0].axvline(df[col].mean(), color='red', linestyle='--', label='Mean')
    axes[0].axvline(df[col].median(), color='green', linestyle='-', label='Median')
    axes[0].set_title(f'{col.title()} Population Density')
    axes[0].legend()

    # Outlier Detection
    sns.boxplot(data=df, x=target, y=col, ax=axes[1], palette='Set2')
    axes[1].set_title(f'Outlier Detection: {col.title()}')

    # Risk Concentration
    sns.violinplot(data=df, x=target, y=col, ax=axes[2], inner='quartile', palette='coolwarm')
    axes[2].set_title(f'Risk Concentration: {col.title()}')
    
    plt.tight_layout()
    plt.show()

# Audit selection of key financial features
numerical_col = ['dti', 'int_rate', 'loan_amount', 'installment', 'total_acc', 'total_payment', 'annual_income_capped']

for col in numerical_col:
    analyze_numeric(df, col, 'defaulted')

# =============================================================================
# 4. BIVARIATE ANALYSIS: MULTICOLLINEARITY & INTERACTION
# =============================================================================
print("\n--- FEATURE CORRELATION MATRIX ---")
correlation_matrix = df[numerical_col].corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0, square=True, linewidths=0.5)
plt.title('Bivariate Correlation Heatmap (Numerical Features)', fontsize=16, fontweight='bold')
plt.show()

# Multicollinearity Audit for Feature Selection
print("\n🔍 MULTICOLLINEARITY WARNINGS (|r| > 0.5):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_value = correlation_matrix.iloc[i, j]
        if abs(corr_value) > 0.5:
            col1, col2 = correlation_matrix.columns[i], correlation_matrix.columns[j]
            label = "REDUNDANT (High Leakage Risk)" if abs(corr_value) > 0.8 else "STRONG INTERACTION"
            print(f"   {col1} ↔ {col2}: r = {corr_value:.3f} | [{label}]")

# =============================================================================
# 5. CATEGORICAL RISK VALIDATION (CHI-SQUARE)
# =============================================================================
def analyze_categorical(df, col, target):
    """
    Quantifies the relationship between qualitative attributes and default 
    probability using Chi-Square Testing of Independence.
    """
    print(f'\n--- CATEGORICAL RISK AUDIT: {col.upper()} ---')
    
    # Segmented Default Rates
    segment_default = (df.groupby(col)[target].mean() * 100).sort_values(ascending=False)
    print(f"Default Probability by {col}:")
    print(segment_default)

    # Statistical Significance Testing
    contingency_table = pd.crosstab(df[col], df[target])
    chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
    print(f"\nCHI-SQUARE STATISTICAL ENGINE:")
    print(f"   Chi-Square Stat: {chi2:.2f}")
    print(f"   P-Value:         {p_value:.4e}")

    if p_value < 0.05:
        print(f"✅ STATISTICALLY SIGNIFICANT: {col} is a high-signal risk predictor.")
    else:
        print(f"❌ STATISTICALLY INSIGNIFICANT: {col} does not reliably discriminate risk.")

    # Risk Volume Visualization
    plt.figure(figsize=(10, 5))
    sns.countplot(data=df, x=col, hue=target, palette='viridis', order=segment_default.index)
    plt.title(f'Loan Volume & Default Frequency by {col.title()}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

categorical_variables = ['emp_length', 'grade', 'home_ownership', 'purpose', 'sub_grade', 'term', 'verification_status']

for col in categorical_variables:
    analyze_categorical(df, col, 'defaulted')

# =============================================================================
# 6. TEMPORAL DYNAMICS: RISK MOMENTUM
# =============================================================================
df['month'] = df['issue_date'].dt.to_period('M')
monthly_metrics = df.groupby('month').agg(Volume=('defaulted', 'count'), Default_Rate=('defaulted', 'mean'))
monthly_metrics['Default_Rate'] *= 100

fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Portfolio Growth (Volume)
monthly_metrics['Volume'].plot(ax=axes[0], marker='o', color='navy', linewidth=2)
axes[0].set_title('Origination Momentum (Loan Volume)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('New Loan Count')
axes[0].grid(True, alpha=0.3)

# Credit Quality Drift (Risk)
monthly_metrics['Default_Rate'].plot(ax=axes[1], marker='o', color='darkred', linewidth=2)
axes[1].axhline(y=monthly_metrics['Default_Rate'].mean(), color='orange', linestyle='--', label='Portfolio Mean')
axes[1].set_title('Credit Quality Drift (Default Rate Trend %)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Default Rate (%)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# =============================================================================
# 7. QUARTERLY MOMENTUM ASSESSMENT
# =============================================================================
print("\n" + "="*40)
print("     FINAL TREND ANALYSIS")
print("="*40)
initial_q_risk = monthly_metrics['Default_Rate'].iloc[:3].mean()
recent_q_risk = monthly_metrics['Default_Rate'].iloc[-3:].mean()

print(f"Historical Baseline (First Quarter): {initial_q_risk:.2f}%")
print(f"Recent Performance (Last Quarter):   {recent_q_risk:.2f}%")

if recent_q_risk > initial_q_risk:
    surge = ((recent_q_risk - initial_q_risk) / initial_q_risk) * 100
    print(f"🚨 WARNING: Risk Leakage detected. Default rate surged by {surge:.1f}% relative to inception.")
else:
    print(f"✅ STABILITY: Portfolio risk is improving or maintaining historical limits.")